In [ ]:
from datasets import load_dataset

# Load a sample of CNN/DailyMail dataset (100 examples)
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test[:100]")
print(dataset[0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

{'article': '(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday

# Step 3: Preprocessing

In [ ]:
 import nltk
 nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
import nltk
nltk.download("punkt")

# Rename for consistency
articles = [example["article"] for example in dataset]
summaries = [example["highlights"] for example in dataset]

import re
import nltk
nltk.download("punkt")
from nltk.tokenize import sent_tokenize

def clean_text(text):
    # Remove multiple spaces, special characters, etc.
    text = re.sub(r"\s+", " ", text)                      # Remove excessive whitespace
    text = re.sub(r"\[[0-9]*\]", "", text)                # Remove [1], [2], etc.
    text = re.sub(r"[^\w\s.,!?;:]", "", text)             # Remove non-alphanumeric chars except punctuation
    text = re.sub(r"\n+", " ", text)                      # Remove line breaks
    text = text.strip()                                   # Trim leading/trailing whitespace
    return text

# Apply cleaning
cleaned_articles = [clean_text(article) for article in articles]
cleaned_summaries = [clean_text(summary) for summary in summaries]

# Optional: truncate for T5-small's input limit
def truncate_text(text, max_sentences=20):
    sentences = sent_tokenize(text)
    return " ".join(sentences[:max_sentences])

truncated_articles = [truncate_text(article) for article in cleaned_articles]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Step 4: BERT (Extractive Summarization) using bert-extractive-summarizer

In [ ]:
from summarizer import Summarizer

bert_model = Summarizer()
bert_summaries = [bert_model(text, min_length=60) for text in truncated_articles]


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Step 5: T5-small (Abstractive Summarization)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

t5_tokenizer = AutoTokenizer.from_pretrained("t5-small")
t5_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

def summarize_t5(text):
    input_ids = t5_tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    output = t5_model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    return t5_tokenizer.decode(output[0], skip_special_tokens=True)

t5_summaries = [summarize_t5(text) for text in truncated_articles]



tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Step 6: Evaluation (ROUGE & BLEU)

In [ ]:
from evaluate import load
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

rouge = load("rouge")
def get_rouge_scores(preds, refs):
    return rouge.compute(predictions=preds, references=refs)

def get_bleu_scores(preds, refs):
    smoothie = SmoothingFunction().method4
    return sum([sentence_bleu([ref.split()], pred.split(), smoothing_function=smoothie) for pred, ref in zip(preds, refs)]) / len(preds)

# Evaluate BERT summaries
bert_rouge = get_rouge_scores(bert_summaries, summaries)
bert_bleu = get_bleu_scores(bert_summaries, summaries)

# Evaluate T5 summaries
t5_rouge = get_rouge_scores(t5_summaries, summaries)
t5_bleu = get_bleu_scores(t5_summaries, summaries)


# Step 7: Print Results

In [ ]:
print("=== BERT Extractive ===")
print("ROUGE:", bert_rouge)
print("BLEU:", bert_bleu)

print("\n=== T5-small Abstractive ===")
print("ROUGE:", t5_rouge)
print("BLEU:", t5_bleu)


=== BERT Extractive ===
ROUGE: {'rouge1': 0.25965522312707323, 'rouge2': 0.0860138492453553, 'rougeL': 0.1842587587288606, 'rougeLsum': 0.21460679389536377}
BLEU: 0.03823947062812369

=== T5-small Abstractive ===
ROUGE: {'rouge1': 0.28330753750370324, 'rouge2': 0.1046304476164924, 'rougeL': 0.20537149356839918, 'rougeLsum': 0.23476265694985696}
BLEU: 0.046713201871764864
